# Zero-dimensional model of sediment accumulation on Polder 32 in Southwest Bangladesh

Import packages

In [2]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
import subprocess
import feather
from tqdm.notebook import tqdm
import multiprocessing as mp
import itertools
import inspect
import shutil

Add project root to path and import project specific definitions

In [6]:
sys.path.append('..')
from src.definitions import get_project_root

root = get_project_root()

Set parameters for the model run. Parameters must be single values (single core) or a list of values (parallel processing). If parallel flag is set to "True", the parallel method will be used.

In [12]:
parallel = False
# Additional parameters needed for parallelization
if parallel is True:
    poolsize = 30
    chunksize = 1

    # Quickly define regular interval for varying parameters.
    # Can also be explicitly set.
    slr = np.round(np.arange(0, 0.0325, 0.0025), 4)
    ssc_factor = np.round(np.arange(0.25, 3.25, 0.25), 2)

# Set model parameters
run_length = 10  # years
dt = '5 min'  # timestep must be given as a timedelta string
slr = 0.003  # yearly rate (m) (0.002 ESLR + 0.001 Tidal Amp)

ssc_factor = 1
ssc = 0.2
# ssc_file = './data/processed/ssc_by_week.csv'
# ssc = pd.read_csv(ssc_file, index_col=0) * ssc_factor

grain_dia = 0.000035  # grain diameter (m)
grain_rho = 2650  # density of quartz
bulk_rho = 1300  # g/L
dP = 0
dO = 0
dM = 0
A = 1
z0 = 1.65

Functions to make tides

In [1]:
def make_combos(**kwargs):
    
    # Initialize arguments list and counter
    var_args = []
    const_args = {}
    n = 0
    
    # Loop through all arguments and extract arguments with multiple values (i.e. arguments that will be varied)
    for key, value in kwargs.items():
        if isinstance(value, (list, tuple, np.ndarray)):
            var_args.append(key)
        else:
            const_args.update({key : value})

    # Initialize output with combination of all multi value arguments
    combos = [dict(zip(var_args, i)) for i in itertools.product(*[eval(x) for x in var_args])]
    
    # Iterate through all combinations of dictionary key:value pairs and add single value arguments
    for combo in combos:
        for key, value in const_args.items():
            combo.update({key : value})
        combo.update({'n' : n}) # combination number for tracking in TQDM
        n = n + 1

    return combos

# Creates a list of tides to be created. Useful when parallelizing and need to
# create tides in advance.

def list_missing_files(wdir, files):
    missing_files = []
    for file in files:
        fp = wdir / arg
        if fp.exists() is False:
            missing_files.append(file)
    

def make_tide_list(slr, run_length, dt):
    wdir = Path.cwd() / 'data' / 'interim' / 'tides'
    if isinstance(slr, list):
        pass
    elif isinstance(slr_list, float):
        slr_list = [slr_list]
    tides_to_make = []
    for rate in slr_list:
        dt_min = int(pd.to_timedelta(dt).total_seconds() / 60)
        fn = ('tides-yr_{0}-dt_{1}-slr_{2:.4f}.feather'
              ).format(run_length, dt_min, rate)
        fp = wdir / fn
        if fp.exists() is False:
            tides_to_make.append(rate)
    return tides_to_make


# Call Rscript and use OCE package to create idealized tide form to
# be used in model


def make_tides(params):
    run_length = params.run_length
    slr = params.slr
    dt = params.dt
    
    R_command = 'Rscript'
    wdir = Path.cwd()
    script_path = wdir / 'scripts' / 'make_tides.R'
    args = [str(run_length), str(dt), '{:.4f}'.format(slr), wdir]
    cmd = [R_command, script_path] + args
    # check_output will run the command and store to result
    x = subprocess.check_output(cmd, universal_newlines=True)
    dt_min = int(pd.to_timedelta(dt).total_seconds() / 60)
    fn = ('tides-yr_{0}-dt_{1}-slr_{2:.4f}.feather'
          ).format(run_length, dt_min, slr)
    fp = wdir / 'data' / 'interim' / 'tides' / fn

    tides = feather.read_dataframe(fp)
    tides = tides.set_index('Datetime')

    return tides

IndentationError: unexpected indent (<ipython-input-1-5ac57c6146a9>, line 14)

Make all tides before running the model.

In [3]:
tides_to_make = make_tide_list(slr, run_length, dt)

if len(tides_to_make) == 0:
    print("All tidal curves already constructed!")
elif len(tides_to_make) == 1:
    make_tides(slr, run_length, dt)
elif len(tides_to_make) >= 2:
    print("Making {0} tidal curves.".format(len(tides_to_make)))
    
    with mp.Pool(poolsize) as pool:
        for new_tide in tqdm(pool.imap_unordered(make_tides,
                                    tides_to_make), total=len(tides_to_make), unit='tidal curves'):
            pass

SyntaxError: EOL while scanning string literal (<ipython-input-3-2016ee092e6e>, line 4)

Create main model function

In [6]:
# Main function for running the model.
# def run_model(tides, ssc, params, n = None)
# Instead of grain_dia, you use params.grain_dia, params.grain_rho, etc.
def run_model(tides, ssc, grain_dia, grain_rho, bulk_rho, dP=0, dO=0, dM=0, A=1, z0=0, n=None):
    global num_runs
    
    # Function that sets the background SSC value given a method.
    def find_ssc(ssc, method, timestamp=None):
        if method == 'constant':
            return ssc
        elif method == 'weekly':
            week = timestamp.week
            ssc = ssc.loc[week].values[0]
            return ssc

    # Return suspended sediment values from a csv of average weekly suspended sediment for P32. When tide is below the
    # the platform (no sedimentation) or the tide is falling (net export), "0" is returned.

    # Calculate the concentration within the water column for a given timestep
    # old method
#     def calc_c(c0, h, h_min_1, dh, c_min_1, z, ws, dt):
#         if (h > z and dh > 0):
#             return (-c_min_1 * ws * dt + (c0-c_min_1) * (h - h_min_1)) / (h_min_1-z) + c_min_1
#         elif (h > z and dh < 0):
#             return (-c_min_1 * ws * dt) / (h_min_1-z) + c_min_1
#         else:
#             return 0
        
    # Method from work with David
    def calc_c(c0, h, h_min_1, dh, c_min_1, z, ws, dt):
        if (h > z and dh > 0):
            return (-c_min_1 * ws * dt + (c0-c_min_1) * (h - h_min_1)) / (h_min_1-z) + c_min_1
        elif (h > z and dh < 0):
            return (-c_min_1 * ws * dt) / (h_min_1-z) + c_min_1
        else:
            return 0
        
#     added by JG
#     global g_calc_c
#     g_calc_c = calc_c

    # Calculate the change in elevation for a given timestep    
    def calc_dz(c, ws, rho, dt):
        return ws * c * dt / rho

    # Add the change of elevation back to the original elevation
    def calc_z(z_min_1, dz_min_1, dO, dP, dM):
        return z_min_1 + dz_min_1 + dO - dP - dM
    
    # Set method to be used for ssc based on ssc input type
    if isinstance(ssc, float):
        ssc_method = 'constant'
    elif isinstance(ssc, pd.DataFrame):
        ssc_method = 'weekly'
    
    # Set Datetime as the index. Feather does not export non-integer indices.
    tides = tides.set_index('Datetime')
    index = tides.index
    dt = index[1] - index[0]
    dt_sec = dt.total_seconds()
    
    # Convert constant rates from yearly to dt
    dO = dO / 8760 / 60 / 60 * dt_sec
    dP = dP / 8760 / 60 / 60 * dt_sec
    dM = dM / 8760 / 60 / 60 * dt_sec
    
    # Assume density and viscosity of water
    fluid_rho = 1000
    fluid_visc = 0.001
    g = 9.8
    
    # Calculate settling velocity using Stokes settling. Considered an upper bound for possible settling velocity.
    ws = (2/9 * (grain_rho - fluid_rho) / fluid_visc) * g * (grain_dia / 2) ** 2
    
    # Initialize numpy arrays for efficiency
    z = np.zeros(len(tides.index))
    z[0] = z0
    h = tides.pressure.values
    dh = np.insert(np.diff(h) / dt_sec,0,np.nan)
    inundated = np.zeros(len(tides.index))
    inundation_depth = np.zeros(len(tides.index))
    C0 = np.zeros(len(tides.index))
    C = np.zeros(len(tides.index))
    dz = np.zeros(len(tides.index))
    SSC = np.zeros(len(tides.index))
    
#     def update_elevation(z, SSC, C0, C, dz, h, dh, dO, dP, dM, A, ssc, bulk_rho, dt_sec, ws, ssc_method, timestamp):
#         z[t] = calc_z(z[t-1], dz[t-1], dO, dP, dM)
#         SSC[t] = find_ssc(ssc, method=ssc_method, timestamp=index[t])
#         C0[t] = calc_c0(h[t], dh[t], ssc, z[t], A)
#         C[t] = calc_c(C0[t], h[t], h[t-1], dh[t], C[t-1], z[t], ws, dt_sec)
#         dz[t] = calc_dz(C[t], ws, bulk_rho, dt_sec)
    
    # For loop to calculate backwards difference approximation.
    # TQDM is a wrapper that shows a status bar while calculating.
    counter = np.arange(1,len(index))
    for t in tqdm(counter, 
                  desc='Run {0} of {1} [PID: {2}]'.format(n, num_runs, os.getpid()), 
                  total=len(index[1:]), 
                  unit='steps'):
        # update_elevation(z = z, SSC = SSC, C0 = C0, C = C, 
        #                  dz = dz, h =  h, dh = dh, dO = dO, dP = dP, dM = dM, 
        #                  A = A, ssc = ssc, bulk_rho = bulk_rho, dt_sec = dt_sec, 
        #                  ws = ws, ssc_method = ssc_method, timestamp = timestamp)
        z[t] = calc_z(z[t-1], dz[t-1], dO, dP, dM)
        SSC[t] = find_ssc(ssc, method=ssc_method, timestamp=index[t])
        C0[t] = SSC[t] * A
        C[t] = calc_c(C0[t], h[t], h[t-1], dh[t], C[t-1], z[t], ws, dt_sec)
        dz[t] = calc_dz(C[t], ws, bulk_rho, dt_sec)
        # Flag if inundated and by how much
        if h[t] - z[t] >= 0:
            inundated[t] = 1
            inundation_depth[t] = h[t] - z[t]
    
    # Create pandas dataframe from numpy arrays of finite difference results
    d = {'h' : h, 'dh' : dh, 'C0' : C0, 'C' : C, 'dz' : dz, 'z' : z, 
         'inundated' : inundated, 'inundation_depth' : inundation_depth}
    df = pd.DataFrame(data=d, index = tides.index)
    
        
    return df

Create helper functions for parallel processing

In [7]:
# Creates combination of parameter values for different scenarios
def make_combos(run_length, dt, slr, ssc_factor, grain_diameter, grain_rho, bulk_rho, dP, dO, dM, A, z0):
    
    # Use inspect function to get all of the input arguments
    args = inspect.getfullargspec(make_combos).args
    
    # Initialize arguments list and counter
    multi_args = []
    n = 0
    
    # Loop through all arguments and extract arguments with multiple values (i.e. arguments that will be varied)
    for arg in args:
        if isinstance(eval(arg), (list, tuple, np.ndarray)):
            multi_args.append(arg)
            
    # Extract the arguments with one value (i.e. arguments that will remain constant)
    single_args = list(set(args) - set(multi_args))
    
    # Create a dictionary of single value arguments
    single_dict = [{'{0}'.format(j) : eval(j)} for j in single_args]
    
    # Initialize output with combination of all multi value arguments
    combos = [dict(zip(multi_args, i)) for i in itertools.product(*[eval(x) for x in multi_args])]
    
    # Iterate through all combinations of dictionary key:value pairs and add single value arguments
    for combo in combos:
        for item in single_dict:
            combo.update(item)
        combo.update({'n' : n}) # combination number for tracking in TQDM
        n = n + 1

    return combos

# Function to be called by parallel function (e.g. imap_unordered). This is necessary because imap only accepts
# one function and one iterable. Using this parser allows the model to be run with multiple iterables package as 
# one tuple
def parallel_parser(in_data):

    n = in_data['n'] # number to explicitly set print line for TQDM. Not working.
    
    # Load tides for a given run_length, dt, and slr from the tide library.
    run_length = in_data['run_length']
    dt = in_data['dt']
    slr = in_data['slr']
    tides = feather.read_dataframe(
        './data/interim/tides/tides-yr_{0}-dt_{1}-slr_{2}.feather'.format(run_length, 
                                                                          int(pd.to_timedelta(dt).total_seconds()/60/60), 
                                                                          '%.4f' % slr))
    # Set Datetime as the index. Feather does not export non-integer indices.
    tides = tides.set_index('Datetime')
    
    # Load weekly ssc data. Original data from OBS sensor deployed at Sutarkhali. Data is in 1-min increments. Developed
    # a model to predict SSC by week of the year (incoming_ssc.R). Output of this script is the weekly SSC loaded here.
    ssc_factor = in_data['ssc_factor'] # scaling factor used to adjust SSC
    ssc_file = './data/processed/ssc_by_week.csv'
    ssc = pd.read_csv(ssc_file, index_col=0) * ssc_factor
    
    # set parameters for model run
    grain_dia = in_data['grain_dia']
    grain_rho = in_data['grain_rho']
    bulk_rho = in_data['bulk_rho']
    dP = in_data['dP']
    dO = in_data['dO']
    dM = in_data['dM']
    A = in_data['A']
    z0 = in_data['z0']
    
    # run model
    # params = in_data["params"]
    # run_model(tides = tides, ssc = ssc, params = params, n = n)
    run_model(tides, ssc, grain_dia, grain_rho, bulk_rho, dP, dO, dM, A, z0, n=n)
    
    # Write results to a feather file
    out_name = 'yr_{0}-slr_{1}-grain_dia_{2}-grain_rho_{3}-bulk_rho_{4}-sscfactor_{5}-dP_{6}-dM_{7}-A_{8}-z0_{9}.feather'.format(run_length, slr, grain_dia, grain_rho, bulk_rho, ssc_factor, dP, dM, A, z0)
    feather.write_dataframe(df.reset_index(), './data/interim/results/{0}'.format(out_name))

    return

In [8]:
# Parallel method
if parallel == True:
    
    # Make combos of parameters
    model_runs = make_combos(run_length, dt, slr, ssc_factor, grain_dia, grain_rho, bulk_rho, dP, dO, dM, A, z0)
    
    # Count number of models to be run
    num_runs = len(model_runs)

    # Initialize pool and run models
    with mp.Pool(poolsize) as pool:
        for result in pool.imap_unordered(parallel_parser, model_runs, chunksize=chunksize):
            pass

# Single core method
elif parallel == False:
    
    # Load tides from tide library
    tides = feather.read_dataframe('./data/interim/tides/tides-yr_{0}-dt_{1}-slr_{2}.feather'.format(
        run_length, int(pd.to_timedelta(dt).total_seconds()/60), '%.4f' % slr))

    # Set number of models to be run
    num_runs = 1

    # run model
    # df = run_model(tides = tides, ssc = ssc, params = params, n = 1)
    df = run_model(tides, ssc, grain_dia, grain_rho, bulk_rho, dP, dO, dM, A, z0, n=1)
    
    # write results to feather file
    out_name = 'yr_{0}-slr_{1}-grain_dia_{2}-grain_rho_{3}-bulk_rho_{4}-sscfactor_{5}-dP_{6}-dM_{7}-A_{8}-z0_{9}.feather'.format(run_length, slr, grain_dia, grain_rho, bulk_rho, ssc_factor, dP, dM, A, z0)
    feather.write_dataframe(df.reset_index(), './data/interim/results/{0}'.format(out_name))

In [9]:
yr1 = np.sum(df['2014-05-17':'2015-05-17'].dz)*100
yr2 = np.sum(df['2015-05-17':'2016-05-17'].dz)*100
yr3 = np.sum(df['2016-05-17':'2017-05-17'].dz)*100
yr4 = np.sum(df['2017-05-17':'2018-05-17'].dz)*100
yr5 = np.sum(df['2018-05-17':'2019-05-17'].dz)*100
yr6 = np.sum(df['2019-05-17':'2020-05-17'].dz)*100
yr10 = np.sum(df['2023-05-17':'2024-05-17'].dz)*100

In [63]:
print(yr1, yr2, yr3, yr4, yr5, yr6, yr10)

-17536132.25389067 -16276255.494442893 -14466147.63276091 -13377189.73421206 -12588479.468255972 -12003088.121481061 -10362995.486001689


In [26]:
df.plot(y = ['h','z'])

In [167]:
yr1 = np.sum(df['2014-05-17':'2015-05-17'].dz)*100
yr2 = np.sum(df['2015-05-17':'2016-05-17'].dz)*100
yr3 = np.sum(df['2016-05-17':'2017-05-17'].dz)*100
yr4 = np.sum(df['2017-05-17':'2018-05-17'].dz)*100
yr5 = np.sum(df['2018-05-17':'2019-05-17'].dz)*100
yr6 = np.sum(df['2019-05-17':'2020-05-17'].dz)*100
yr7 = np.sum(df['2020-05-17':'2021-05-17'].dz)*100
yr8 = np.sum(df['2021-05-17':'2022-05-17'].dz)*100
yr9 = np.sum(df['2022-05-17':'2023-05-17'].dz)*100
yr10 = np.sum(df['2023-05-17':'2024-05-17'].dz)*100
dfz = pd.DataFrame((yr1, yr2, yr3, yr4, yr5, yr6, yr7, yr8, yr9, yr10))

In [45]:
df.to_csv('out.csv')

In [66]:
out = feather.read_dataframe('data/interim/tides/tides-yr_1-dt_5-slr_0.0020.feather')

In [67]:
out.to_csv('tides.csv')

PosixPath('..')